# Clase Práctica #1 (Compilación)

A lo largo del curso estaremos implementando un compilador para el lenguaje de programación `HULK` (*Havana University Language for Kompilers*), paso a paso, introduciendo nuevas características del lenguaje o mejorando la implementación de otras características a medida que vamos descubriendo las técnicas fundamentales de la teoría de lenguajes y la compilación.

El objetivo de esta clase es construir un evaluador de expresiones "a mano", usando los recursos que tenemos hasta el momento. Para ello vamos a comenzar con una versión de `HULK` muy sencilla, un lenguaje de expresiones aritméticas.

## Evaluador de expresiones

Definiremos a continuación este lenguaje de manera informal:

Un programa en `HULK` consta de una secuencia de expresiones. Cada expresión está compuesta por:

- números (con coma flotante de 32 bits), 
- operadores `+ *` con el orden operacional, y
- paréntesis `(` y `)`

### Análisis lexicográfico

Comenzaremos construyendo un prototipo bien simple, donde asumiremos que en la expresión hay espacios en blanco entre todos los elementos, de modo que el *lexer* se reduce a dividir por espacios. Luego iremos adicionando elementos más complejos.

El siguiente método devuelve una lista de *tokens*, asumiendo que la expresión solo tiene números, operadores y paréntesis, separados por espacios en blanco.

In [17]:
def tokenize(text):
    """
    Returns the set of tokens. At this point, simply splits by 
    spaces and converts numbers to `float` instances.
    """     
    return [int(item) if item.isnumeric() else item for item in text.split()]
    
assert tokenize('5 + 6 * 9') == [5, '+', 6, '*', 9]
print(tokenize('5 + 6 * 9'))

[5, '+', 6, '*', 9]


### Análisis sintáctico y evaluación

Una vez que tenemos los *tokens*, solo nos queda evaluar la expresión. Usaremos para ello una idea simple, pero bien útil: evaluaremos recursivamente la expresión descendiendo por los distintos niveles de precedencia.

Toda expresión del lenguaje puede ser vista como una suma de _términos_, donde cada uno de estos "_términos_" se descompone a su vez en operaciones de multiplicación entre _factores_. Incluso si no hay operadores `+` en toda la expresión queda claro que esta idea es válida puesto que estaríamos en presencia de una expresión formada por un solo _término_. Los _factores_ del lenguaje son todos unidades atómicas: por ahora solo números y expresiones complejas envueltas entre paréntesis. Nótese que el uso de paréntesis permite reiniciar el descenso por los niveles de precedencia (regresar a los niveles más altos).

In [18]:
# E -> T + T 
# T -> F * F
# F -> num | (E)

# E -> E + E | Term
#Term -> Factor * Factor | Factor
#Factor -> num|(Exp)

# These lambda expressions map from operators to actual executable code
operations = {
    '+': lambda x,y: x + y,
    '*': lambda x,y: x * y,
}

In [19]:
# Some util classes and methods

class ParsingError(Exception):
    """
    Clase base para todas las excepciones del analisis sintactico(parsing)
    Base class for all parsing exceptions.
    """
    pass

class BadEOFError(ParsingError):

    """
    Unexpected EOF error.
    """
    
    def __init__(self):
        ParsingError.__init__(self, "Unexpected EOF")
        
class UnexpectedToken(ParsingError):
    
    """
    Error
    Unexpected token error.
    """
    
    def __init__(self, token, i):
        ParsingError.__init__(self, f'Unexpected token: {token} at {i}')
        
class MissingCloseParenthesisError(ParsingError):
    """

    Missing ')' token error.
    """
    
    def __init__(self, token, i):
        ParsingError.__init__(self, f'Expected ")" token at {i}. Got "{token}" instead')
        
class MissingOpenParenthesisError(ParsingError):
    """
    Missing '(' token error.
    """
    
    def __init__(self, token, i):
        ParsingError.__init__(self, f'Expected "(" token at {i}. Got "{token}" instead')

def get_token(tokens, i, error_type=BadEOFError):
    """
    Retorna tokens[i] if 'i' esta en rango . si no lanza un ParsingError
    Returns tokens[i] if 'i' is in range. Otherwise, raises ParsingError exception.
    """
    
    try:
        return tokens[i]
    except IndexError:
        raise error_type()

In [20]:
#La siguiente gramatica es ambigua puesto que existen dos parse izq para una misma cadena 
# E -> E + E | E * E | num | (E)

# Las precedencias de esta gramatica no es correcta
# E -> T + T
# T -> F * F
# F -> num | (E)
# La siguiente gramatica tiene las precedencias bien pero asocia mal
# por ejemplo 1-1+1 = 1 sin embargo si asocia a la derecha entonces se hace 1-(1+1) != (1-1)+1
# E -> T + E | T
# T -> F * T | F
# F -> int | (E)
# La siguiente gramatica tiene las presedencias correctamente y asocia correctamante
#  E -> E + T| T
#  T -> T * F | F
#  F -> int | (E)

# Esta gramatica es correcta sin embargo crea un loop en el parser puesto que E -> E + T -> E + T + T ...
# Entonce para un parser la recursividad izquierda es un problema
# Como elimiarla
# E-> TX
# X -> +TX|epsilon
# T -> FY
# Y -> *FY|epsilon
#  F -> int | (E)
# Por ultimo queremos anadir funciones elementales con multiples argumentos.
# E -> TX
# X -> +TX |epsilon
# T -> FY
# Y -> *FY |epsilon
# F -> int | (E) |srt(A)
# A -> EC
# C -> ,EC  | epsilon

operations = {
    '+': lambda x, y: x + y,
    '-': lambda x, y: x - y,
    '*': lambda x, y: x * y,
    '/': lambda x, y: x / y,
}
import math

functions = {
    'sin': lambda x: math.sin(x),
    'cos': lambda x: math.cos(x),
    'tan': lambda x: math.tan(x),
    'log': lambda x,y: math.log(x, y),
    'sqrt': lambda x: math.sqrt(x),
}




def evaluate(tokens):
    """
    Evaluates an expression recursively.
    """
    try:
        i, value = parse_expression(tokens, 0)
        assert i == len(tokens)
        return value
    except ParsingError as error:
        print(error)
        return None


expr_operations = ['+', '-']


def parse_expression(tokens, i):
    i, term = parse_term(tokens, i)
    return parse_X(term, tokens, i)

def parse_X(value, tokens, i):
    if i < len(tokens):
        token = get_token(tokens, i)
        if token in expr_operations:
            i, term = parse_term(tokens, i+1)
            operation = operations[token]
            value = operation(value, term)
            i, value = parse_X(value, tokens, i)

    return i, value


term_operations = ['*', '/']


def parse_term(tokens, i):
    i, factor = parse_factor(tokens, i)
    return parse_Y(factor, tokens, i)


def parse_Y(value, tokens, i):
    if i < len(tokens):
        token = get_token(tokens, i)
        if token in term_operations:
            i, factor = parse_factor(tokens, i+1)
            operation = operations[token]
            value = operation(value, factor)
            i, value = parse_Y(value, tokens, i)
    return i, value




def parse_factor(tokens, i):
    token = opert_fun =  get_token(tokens, i)

    if opert_fun in functions.keys():
        token = get_token(tokens, i+1)
        if token == "(":
            
            i,args = parse_Arg(tokens,i+2)

            try:
                func = functions[opert_fun]
                value = func(*args)
            except:
                raise UnexpectedToken(tokens[i],i)
            
            token = get_token(tokens, i)
            if token != ")":
                raise MissingCloseParenthesisError(tokens[i], i)
            return i+1,value

        else:
            raise MissingOpenParenthesisError(tokens[i], i)

    if token == "(":
        i, expression = parse_expression(tokens, i+1)
        if get_token(tokens, i) != ")":
            raise MissingCloseParenthesisError(tokens, i)
        return i+1, expression
    else:
        if isinstance(token, (int, float)):
            return i+1, token
        else:
            raise UnexpectedToken(tokens[i], i)

def parse_Arg(tokens, i):
    args = []
    i,value = parse_expression(tokens,i)
    args.append(value)
    i,args = parse_C(args,tokens,i)
    return i,args
    
def parse_C(args,tokens,i):
    if i < len(tokens):
        token = get_token(tokens, i)
        if token == ",":
            i, value = parse_expression(tokens, i+1)
            args.append(value)
            i,args  = parse_C(args,tokens, i)
        return i,args



print(evaluate(tokenize('5 + 6 * 9')))
assert evaluate(tokenize('5 + 6 * 9')) == 59.
assert evaluate(tokenize('( 5 + 6 ) * 9')) == 99.
assert evaluate(tokenize('( 5 + 6 ) + 1 * 9 + 2')) == 22.

59


## Adicionando constantes

Agreguemos constantes numéricas al lenguaje `HULK` Para ello, simplemente añadiremos un diccionario con todas las constantes disponibles, que usaremos durante la tokenización. Nótese que solo es necesario modificar el _lexer_ para añadir este rasgo al lenguaje.

In [21]:
constants = {
    'pi': 3.14159265359,
    'e': 2.71828182846,
    'phi': 1.61803398875,
}

In [22]:
def tokenize(expr):
    """
    Returns the set of tokens. At this point, simply splits by 
    spaces and converts numbers to `float` instances.
    Replaces constants.
    """
    return [int(item) if item.isnumeric()  else constants[item] if item in constants.keys() else item for item in expr.split()]

assert tokenize('2 * pi') == [2.0, '*', 3.14159265359]

tokenize('2 * pi') 

assert evaluate(tokenize('2 * pi')) == 6.28318530718

## Adicionando operadores `-` y `/`

- **Restricción:** No utilizar ciclos!!!

In [23]:
# These lambda expressions map from operators to actual executable code
operations = {
    '+': lambda x,y: x + y,
    '-': lambda x,y: x - y,
    '*': lambda x,y: x * y,
    '/': lambda x,y: x / y,
}

In [24]:
# def parse_expression(tokens, i):
#     # Insert your code here ...
#     pass

        
# def parse_term(tokens, i):
#     # Insert your code here ...
#     pass



In [25]:
assert evaluate(tokenize('1 - 1 + 1')) == 1
assert evaluate(tokenize('8 / 4 / 2')) == 1

## Adicionando funciones elementales

Agreguemos funciones elementales `sin`, `cos`, `tan`, `log`, `sqrt`, etc. El llamado a funciones se hará en notación prefija, comenzando por el nombre de la función y seguido, entre paréntesis, por los argumentos, que estarán separados entre sí por _comas_.

Para las funciones elementales haremos algo similar a las constantes, pero en vez de a la hora de tokenizar, las reemplazaremos a la hora de evaluar, pues necesitamos evaluar recursivamente los argumentos de la función. Empezaremos por garantizar que nuestro tokenizador que es capaz de reconocer expresiones con funciones elementales de más de un argumento, en caso de no ser así es necesario arreglarlo.

In [26]:
assert tokenize('log ( 64 , 1 + 3 )') == ['log', '(', 64.0, ',', 1.0, '+', 3.0, ')']

Adicionaremos entonces un diccionario con todas las funciones elementales que permitiremos.

In [27]:
import math

functions = {
    'sin': lambda x: math.sin(x),
    'cos': lambda x: math.cos(x),
    'tan': lambda x: math.tan(x),
    'log': lambda x,y: math.log(x, y),
    'sqrt': lambda x: math.sqrt(x),
}

Por último, modificaremos el método `evaluate` para que use las funciones elementales. Recordemos que los argumentos están separados por el token _coma_ (`,`) y que cada uno puede a su vez tener sub-expresiones que consistan también en llamados a funciones.

In [28]:
# def parse_factor(tokens, i):
#     # Insert your code here ...
#     # (You may copy and modify your previous implementation of 'parse_factor')
#     pass
    
assert evaluate(tokenize('log ( 64 , 1 + 3 )')) == 3.0